In [3]:
from datasets import load_dataset

train = load_dataset(
    "parquet",
    data_files="/home/fanhaojie/newCode/data/uld_loss_Mistral-7B-Instruct-v0.2-qed/train-00000-of-00001-00ee1a1595974b81.parquet"
)

print(train)

/home/fanhaojie/anaconda3/envs/LLMs/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'answers', 'answers_generated'],
        num_rows: 7009
    })
})


In [4]:
validation = load_dataset(
    "parquet",
    data_files="/home/fanhaojie/newCode/data/uld_loss_Mistral-7B-Instruct-v0.2-qed/validation-00000-of-00001-d14b657647a15533.parquet"
)

print(validation)


DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'answers', 'answers_generated'],
        num_rows: 610
    })
})


In [1]:
def preprocess(example):
    input_text = f"{example['context']}\n问题：{example['question']}"
    answer = example.get('answers', [""])
    if isinstance(answer, list) and len(answer) > 0:
        target_text = str(answer[0]) if answer[0] is not None else ""
    else:
        target_text = ""
    model_inputs = tokenizer(
        input_text,
        max_length=512,
        truncation=True,
        padding='max_length'  # 强制补齐
    )
    labels = tokenizer(
        target_text,
        max_length=64,
        truncation=True,
        padding='max_length'  # 强制补齐
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-560m")
# 处理数据
processed_train = train["train"].map(preprocess)
processed_validation = validation["train"].map(preprocess)
# 构造 DatasetDict
processed_dataset = {"train": processed_train, "validation": processed_validation}
print(processed_dataset)
print(processed_dataset["train"][0])

{'train': Dataset({
    features: ['title', 'context', 'question', 'answers', 'answers_generated', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 7009
}), 'validation': Dataset({
    features: ['title', 'context', 'question', 'answers', 'answers_generated', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 610
})}
{'title': 'Rory McCann', 'context': "Rory McCann ( born 24 April 1969 ) is a Scottish actor , best known for portraying Sandor `` The Hound '' Clegane on the HBO series Game of Thrones and Lurch in Edgar Wright 's crime - comedy Hot Fuzz .", 'question': 'who plays the character dog in game of thrones', 'answers': [{'end': 11, 'start': 0, 'string': 'Rory McCann'}], 'answers_generated': 'Rory McCann (plays "The Hound")', 'input_ids': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3